In [ ]:
!pip install langchain_google_genai pypdf2 google.generativeai langchain_community langchain faiss-cpu


In [ ]:

from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

d:\work\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
# loader=PyPDFLoader(r'./files/medical_treatment.pdf')
# docs=loader.load()
# docs[:5]

[Document(metadata={'source': './files/medical_treatment.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': './files/medical_treatment.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': './files/medical_treatment.pdf', 'page': 2}, page_content='2022CURRENT\nMedical \nDiagnosis\n& T reatmentS\nI X T Y - F I R S T  E D I T I O NEdited \nby\nMaxine A. Papadakis, MDProf\nessor of Medicine, Emeritus\nDepartment of Medicine\nUniversity of California, San FranciscoStephen \nJ. McPhee, MDProf\nessor of Medicine, Emeritus\nDivision of General Internal Medicine\nDepartment of Medicine\nUniversity of California, San FranciscoMichael \nW. Rabow, MDProf\nessor of Medicine and Urology\nDivision of Palliative Medicine\nDepartment of Medicine\nUniversity of California, San Francisco\nAssociate EditorKenneth \nR. McQuaid, MDProf\nessor of Medicine\nDepartment of Medicine\nUniversity of California, San Francisco\nWith Associate Authors\nNew York Chicago San Francisco Athens

In [10]:
EMBEDDING_MODEL = "models/embedding-001"
FAISS_INDEX_NAME = "faiss_index"
# PDF_DIRECTORY = os.path.join(os.path.dirname(os.path.realpath(__file__)), "files")

# Utility functions
def get_pdf_text(pdf_docs):
    return "".join(page.page_content for page in pdf_docs)

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    return text_splitter.split_text(text)

def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local(FAISS_INDEX_NAME)

def convert_pdfs_to_vectors(dir_path):
    if not os.path.isdir(dir_path):
        raise ValueError(f"The directory '{dir_path}' does not exist.")
    
    for file_name in os.listdir(dir_path):
        if file_name.endswith(".pdf"):
            file_path = os.path.join(dir_path, file_name)
            print(f"Processing {file_name}...")
            loader = PyPDFLoader(file_path)
            documents = loader.load()
            text_chunks = get_text_chunks(get_pdf_text(documents))
            get_vector_store(text_chunks)

In [11]:
convert_pdfs_to_vectors('./files')

Processing medical_treatment.pdf...


In [ ]:
# def get_pdf_text(pdf_docs):
#     text=""
#     for page in pdf_docs:
#         text+= page.page_content
#     return  text



# def get_text_chunks(text):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
#     chunks = text_splitter.split_text(text)
#     return chunks


# def get_vector_store(text_chunks):
#     embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
#     vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
#     vector_store.save_local("faiss_index")

In [ ]:
# text = get_pdf_text(docs)
# text_chuncks = get_text_chunks(text)
# print(text_chuncks[:5])
# get_vector_store(text_chuncks)

['2022CURRENT\nMedical \nDiagnosis\n& T reatmentS\nI X T Y - F I R S T  E D I T I O NEdited \nby\nMaxine A. Papadakis, MDProf\nessor of Medicine, Emeritus\nDepartment of Medicine\nUniversity of California, San FranciscoStephen \nJ. McPhee, MDProf\nessor of Medicine, Emeritus\nDivision of General Internal Medicine\nDepartment of Medicine\nUniversity of California, San FranciscoMichael \nW. Rabow, MDProf\nessor of Medicine and Urology\nDivision of Palliative Medicine\nDepartment of Medicine\nUniversity of California, San Francisco\nAssociate EditorKenneth \nR. McQuaid, MDProf\nessor of Medicine\nDepartment of Medicine\nUniversity of California, San Francisco\nWith Associate Authors\nNew York Chicago San Francisco Athens London Madrid Mexico City\nMilan New Delhi Singapore Sydney Torontoa LANGE medical bookCMDT2022_FM_pi-xx.indd \n1 02/07/21 7:10 PMCopyright © 2022 by McGraw Hill. All rights reserved. Except as permitted under the United States Copyright Act of 1976,\nno part of this publ

In [6]:
def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, 
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-8b",
                             temperature=0.3)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

In [7]:
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    
    response = chain(
        {"input_documents":docs, "question": user_question},
        return_only_outputs=True
    )

    return response

In [8]:
result = user_input("I have fever, cold, and some rashing on my skin. what disease can i have?")

print(result['output_text'])

C:\Users\ycham\AppData\Local\Temp\ipykernel_2788\1577411899.py:15: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
C:\Users\ycham\AppData\Local\Temp\ipykernel_2788\23944662.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain(


The provided text lists several diseases that can cause fever, a runny nose (coryza), and a rash.  However, it does *not* give a definitive answer to which disease you might have.  To determine the cause, you need to see a doctor.  They can assess your symptoms, medical history, and possibly perform some tests to make a diagnosis.  

The diseases mentioned that *could* be possibilities include:

* **Measles (rubeola):**  Characterized by fever, coryza, conjunctivitis, and a maculopapular rash that starts on the head and neck and spreads downward.
* **Varicella (chickenpox):**  A rash that evolves rapidly from macules to papules, vesicles, and crusts, with lesions often present in various stages simultaneously.
* **Rickettsialpox:**  Can have a rash that resembles chickenpox, though the text notes that rickettsialpox lesions are typically papulovesicular, not strictly vesicular.
* **Other viral infections:**  The text mentions several other viral infections that can cause a rash, includ